In [0]:
from pyspark import SparkConf,SparkContext
conf = SparkConf().setAppName("students_project")
context = SparkContext.getOrCreate(conf=conf)

In [0]:
text = context.textFile("/FileStore/tables/StudentData.csv")

In [0]:
text_header = text.first()
text = text.filter(lambda x: x != text_header).map(lambda x: x.split(','))

In [0]:
print("There are a total of", text.count(), "students in the data")

There are a total of 1000 students in the data


In [0]:
marks_count_per_gender = text.map(lambda x: (x[1],int(x[5]))).reduceByKey(lambda x,y: sum([x,y])).map(lambda x: f'Total {x[0]} Marks: {x[1]}')
marks_count_per_gender.collect()

Out[169]: ['Total Female Marks: 29636', 'Total Male Marks: 30461']

In [0]:
students_passed = text.filter(lambda x: int(x[5]) >= 50)
students_failed = text.filter(lambda x: int(x[5]) < 50)
print(students_passed.count(), "students passed.", students_failed.count(), "students failed.")

644 students passed. 356 students failed.


In [0]:
total_students_per_course = text.map(lambda x: (x[3],1)).reduceByKey(lambda x,y: sum([x,y])).map(lambda x: x[0] + ': ' + str(x[1]))
print("Students enrolled per course:", total_students_per_course.collect())

Students enrolled per course: ['DB: 157', 'Cloud: 192', 'PF: 166', 'MVC: 157', 'OOP: 152', 'DSA: 176']


In [0]:
total_marks_per_course = text.map(lambda x: (x[3],int(x[5]))).reduceByKey(lambda x,y: sum([x,y])).map(lambda x: x[0] + ': ' + str(x[1]))
print("Total Marks per course:", total_marks_per_course.collect())

Total Marks per course: ['DB: 9270', 'Cloud: 11443', 'PF: 9933', 'MVC: 9585', 'OOP: 8916', 'DSA: 10950']


In [0]:
average_marks_per_course = text.map(lambda x: (x[3],(int(x[5]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x: x[0] + ': ' + str(round(x[1][0]/x[1][1],2)))
print("Average Marks Per Course", average_marks_per_course.collect())

Average Marks Per Course ['DB: 59.04', 'Cloud: 59.6', 'PF: 59.84', 'MVC: 61.05', 'OOP: 58.66', 'DSA: 62.22']


In [0]:
maximum_mark_per_course = text.map(lambda x: (x[3],int(x[5]))).reduceByKey(lambda x,y: max([x,y])).map(lambda x: 'Max for Course ' + x[0] + ': ' + str(x[1]))
maximum_mark_per_course.collect()

Out[174]: ['Max for Course DB: 98',
 'Max for Course Cloud: 99',
 'Max for Course PF: 99',
 'Max for Course MVC: 99',
 'Max for Course OOP: 99',
 'Max for Course DSA: 99']

In [0]:
minimum_mark_per_course = text.map(lambda x: (x[3],int(x[5]))).reduceByKey(lambda x,y: min([x,y])).map(lambda x: 'Min for Course ' + x[0] + ': ' + str(x[1]))
minimum_mark_per_course.collect()

Out[175]: ['Min for Course DB: 20',
 'Min for Course Cloud: 20',
 'Min for Course PF: 20',
 'Min for Course MVC: 22',
 'Min for Course OOP: 20',
 'Min for Course DSA: 20']

In [0]:
avg_age_genders = text.map(lambda x: (x[1],(int(x[0]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0],round(x[1][0]/x[1][1],2))).map(lambda x: 'Average ' + x[0] + ' Age: ' + str(x[1]))